In [1]:
import librosa
import librosa.filters
import numpy as np

#def _process_utterance (preprocessor.py)
wav_path = 'C:/Users/o/Desktop/audio_project/Tacotron-2/LJSpeech-1.1/wavs/LJ001-0002.wav'
wav = librosa.core.load(wav_path, sr=22050)[0]
wav

array([-0.21065567, -0.33885282, -0.1524703 , ...,  0.00181212,
        0.00198035,  0.00288754], dtype=float32)

### trim_silence(audio.py)

In [6]:
librosa.effects.trim(wav,top_db=23,frame_length=512,hop_length=128)[0].shape

(208640,)

In [3]:
def trim_silence(wav): #audio.py
    librosa.effects.trim(wav,top_db=23,frame_length=512,hop_length=128)[0]
    
trim_silence(wav)

None


In [4]:
import librosa
import librosa.filters
import numpy as np
import tensorflow as tf
from scipy import signal
from scipy.io import wavfile

C:\Users\o\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### get_hop_size(hparams) (audio.py)

In [5]:
def get_hop_size():
    hop_size = 275
    if hop_size is None:
        assert hparams.frame_shift_ms is not None
        hop_size = int(None / 1000 * 22050)
    return hop_size

### melspectrogram(audio.py)

In [6]:
def _lws_processor():
    import lws
    return lws.lws(2048, get_hop_size(), fftsize=1100, mode="speech")

In [7]:
def _stft(y): #short time fourier transform
    #if hparams.use_lws:
    #    return _lws_processor(hparams).stft(y).T
    #else:
    return librosa.stft(y=y, n_fft=2048, hop_length=get_hop_size(), win_length=1100)

In [8]:
def preemphasis(wav, k, preemphasize=True):
    if preemphasize:
        return signal.lfilter([1, -k], [1], wav)
    return wav

In [9]:
#def melspectrogram(wav,hparams): (audio.py)
D = _stft(preemphasis(wav,0.97,True)) #(1025, 802)

In [10]:
def _amp_to_db(x):
    min_level = np.exp( -100 / 20 * np.log(10)) 
    return 20 * np.log10(np.maximum(min_level, x))

In [11]:
def _build_mel_basis():
    assert 7600 <= 22050 // 2
    return librosa.filters.mel(22050, 2048, n_mels=80,
                               fmin=55, fmax=7600) #n_mels = #Number of mel-spectrogram channels and local conditioning dimensionality
#fmin : 이값이 55면 male 95면 should help taking off noise

In [12]:
_mel_basis = None

def _linear_to_mel(spectogram):
    global _mel_basis
    if _mel_basis is None:
        _mel_basis = _build_mel_basis()
    return np.dot(_mel_basis, spectogram) #_mel_basis, spectrogram dot product

In [13]:
#def melspectrogram(wav,hparams): (audio.py)
D = _stft(preemphasis(wav,0.97,True)) #(1025, 802)
S = _amp_to_db(_linear_to_mel(np.abs(D))) - 20 #(80,802)
S

array([[-65.17639178, -63.81579223, -65.33661227, ..., -92.7181426 ,
        -79.31536568, -73.70615071],
       [-60.07190905, -55.73654628, -56.7799492 , ..., -80.7047733 ,
        -78.34903129, -76.53380932],
       [-59.90354292, -55.91014862, -57.02379754, ..., -79.56741829,
        -80.71872171, -86.96110007],
       ...,
       [-57.58460998, -60.24204231, -59.04554332, ..., -84.81655836,
        -88.366323  , -95.3308984 ],
       [-57.45054019, -61.84319912, -68.06492264, ..., -88.55946752,
        -90.07159373, -95.92540955],
       [-56.14826313, -60.42914531, -68.63958203, ..., -91.37407124,
        -92.26017924, -93.87807198]])

In [79]:
def melspectrogram(wav):
    return S
    
melspectrogram(wav)

array([[-65.17639178, -63.81579223, -65.33661227, ..., -92.7181426 ,
        -79.31536568, -73.70615071],
       [-60.07190905, -55.73654628, -56.7799492 , ..., -80.7047733 ,
        -78.34903129, -76.53380932],
       [-59.90354292, -55.91014862, -57.02379754, ..., -79.56741829,
        -80.71872171, -86.96110007],
       ...,
       [-57.58460998, -60.24204231, -59.04554332, ..., -84.81655836,
        -88.366323  , -95.3308984 ],
       [-57.45054019, -61.84319912, -68.06492264, ..., -88.55946752,
        -90.07159373, -95.92540955],
       [-56.14826313, -60.42914531, -68.63958203, ..., -91.37407124,
        -92.26017924, -93.87807198]])

#### _normalize(audio.py - melspectrogram에서 쓰임)

In [25]:
#allow_clipping_in_normalization
#symetric_mels인 경우
def _normalize(S):
    #allow_clipping_in_normalization
    #symetric_mels인 경우
    return np.clip((2 * 4) * ((S - (-100)) / 100) -4 ,-4, 4) #최소-4, 최대 4로 정규화(넘는건 다 잘라버림)
#(2 * 4) * ((S + 100) / (100)) - 4.을 대상으로 
_normalize(S)

array([[-1.21411134, -1.10526338, -1.22692898, ..., -3.41745141,
        -2.34522925, -1.89649206],
       [-0.80575272, -0.4589237 , -0.54239594, ..., -2.45638186,
        -2.2679225 , -2.12270475],
       [-0.79228343, -0.47281189, -0.5619038 , ..., -2.36539346,
        -2.45749774, -2.95688801],
       ...,
       [-0.6067688 , -0.81936338, -0.72364347, ..., -2.78532467,
        -3.06930584, -3.62647187],
       [-0.59604322, -0.94745593, -1.44519381, ..., -3.0847574 ,
        -3.2057275 , -3.67403276],
       [-0.49186105, -0.83433162, -1.49116656, ..., -3.3099257 ,
        -3.38081434, -3.51024576]])

In [54]:
#allow_clipping_in_normalization
#symmetric_mels가 아닌경우
np.clip(4 * ((S + 100) / (100)), 0, 4.) #4 * ((S + 100) / (100))를 대상으로 0과 4 사이로(0보다 작으면 0, 4보다 크면 4)

array([[1.39294433, 1.44736831, 1.38653551, ..., 0.2912743 , 0.82738537,
        1.05175397],
       [1.59712364, 1.77053815, 1.72880203, ..., 0.77180907, 0.86603875,
        0.93864763],
       [1.60385828, 1.76359406, 1.7190481 , ..., 0.81730327, 0.77125113,
        0.521556  ],
       ...,
       [1.6966156 , 1.59031831, 1.63817827, ..., 0.60733767, 0.46534708,
        0.18676406],
       [1.70197839, 1.52627204, 1.27740309, ..., 0.4576213 , 0.39713625,
        0.16298362],
       [1.75406947, 1.58283419, 1.25441672, ..., 0.34503715, 0.30959283,
        0.24487712]])

In [55]:
#not allow_clipping_in_normalization
#symmetric_mels인 경우
#(2 * hparams.max_abs_value) * ((S - hparams.min_level_db) / (-hparams.min_level_db)) - hparams.max_abs_value
(2 * 4.) * ((S + 100) / (100)) - 4.

array([[-1.21411134, -1.10526338, -1.22692898, ..., -3.41745141,
        -2.34522925, -1.89649206],
       [-0.80575272, -0.4589237 , -0.54239594, ..., -2.45638186,
        -2.2679225 , -2.12270475],
       [-0.79228343, -0.47281189, -0.5619038 , ..., -2.36539346,
        -2.45749774, -2.95688801],
       ...,
       [-0.6067688 , -0.81936338, -0.72364347, ..., -2.78532467,
        -3.06930584, -3.62647187],
       [-0.59604322, -0.94745593, -1.44519381, ..., -3.0847574 ,
        -3.2057275 , -3.67403276],
       [-0.49186105, -0.83433162, -1.49116656, ..., -3.3099257 ,
        -3.38081434, -3.51024576]])

In [ ]:
#non symmetric_mels인 경우
hparams.max_abs_value * ((S - hparams.min_level_db) / (-hparams.min_level_db))

### def inv_linear_spectrogram(linear_spectrogram, haprams) (audio.py)

convert linear spectrogram to waveform using librosa

#### _denormalize(D,haparams)

In [60]:
#allow_clipping_in_normalization
#symmetric_mels 라면
((((np.clip(D, -4.,4.) + 4.) * 100 / (2 * 4.)) - 100))

array([-47.144463+0.j, -46.950703+0.j, -46.90126 +0.j, -47.282337+0.j,
       -48.45782 +0.j, -49.72403 +0.j, -49.953033+0.j, -49.930946+0.j,
       -50.044094+0.j, -50.061073+0.j, -50.06622 +0.j, -50.346436+0.j,
       -51.12453 +0.j, -51.20158 +0.j, -50.260864+0.j, -49.719704+0.j,
       -49.614727+0.j, -49.589283+0.j, -49.597942+0.j, -49.65122 +0.j,
       -49.852745+0.j, -49.907814+0.j, -49.891407+0.j, -49.886547+0.j,
       -49.889893+0.j, -49.89051 +0.j, -49.936695+0.j, -49.943817+0.j,
       -49.888535+0.j, -49.92501 +0.j, -49.929047+0.j, -49.943226+0.j,
       -49.944817+0.j, -50.12447 +0.j, -50.31583 +0.j, -50.379482+0.j,
       -50.18011 +0.j, -49.903427+0.j, -50.1257  +0.j, -50.49699 +0.j,
       -50.792557+0.j, -50.794556+0.j, -50.4142  +0.j, -50.15916 +0.j,
       -50.097652+0.j, -50.045223+0.j, -50.039307+0.j, -50.05754 +0.j,
       -50.024998+0.j, -49.978004+0.j, -49.94908 +0.j, -49.97777 +0.j,
       -50.055897+0.j, -50.022526+0.j, -49.98828 +0.j, -49.9225  +0.j,
      

In [61]:
#not symmetric_mels라면
((np.clip(D, 0, 4.) * 100 / 4.) - 100)

array([[ -94.28892 +0.0000000e+00j,  -93.901405+0.0000000e+00j,
         -93.80252 +0.0000000e+00j, ..., -100.      +0.0000000e+00j,
        -100.      +0.0000000e+00j,  -99.81981 +0.0000000e+00j],
       [-100.      +0.0000000e+00j, -100.      +0.0000000e+00j,
        -100.      +0.0000000e+00j, ...,  -99.98457 +6.8515912e-03j,
         -99.969505-5.5518830e-03j, -100.      +0.0000000e+00j],
       [ -97.77495 +5.8956114e-16j,  -97.08311 -3.4292546e-01j,
         -97.37198 +2.1212284e-01j, ..., -100.      +0.0000000e+00j,
        -100.      +0.0000000e+00j, -100.      +0.0000000e+00j],
       ...,
       [ -93.31499 -1.1153992e-15j,  -99.606285+3.3190615e+00j,
        -100.      +0.0000000e+00j, ...,  -99.99993 -4.1060146e-05j,
        -100.      +0.0000000e+00j, -100.      +0.0000000e+00j],
       [-100.      +0.0000000e+00j,  -97.78347 -2.4972162e+00j,
         -99.996796-1.5544059e-03j, ..., -100.      +0.0000000e+00j,
         -99.99998 -5.4831838e-04j,  -99.98628 +1.1344699e-02j]

In [ ]:
#not allow_clipping_in_normalization
#symmetric이라면
#(((D + hparams.max_abs_value) * -hparams.min_level_db / (2 * hparams.max_abs_value)) + hparams.min_level_db)
(((D + 4.) * 100 / (2 * 4.)) - 100)

#not symmetric이라면
#((D * -hparams.min_level_db / hparams.max_abs_value) + hparams.min_level_db)
((D * 100 / 4.) - 100)

In [ ]:
#if signal_normalization = True
#Whether to normalize mel spectrograms to some predefined range (following below parameters)
D = _denormalize(linear_spectrogram)

### inv_mel_spectrogram(mel_spectrogram, hparams)

Converts mel spectrogram to waveform using librosa

In [67]:
def inv_mel_spectrogram(mel_spectrogram, hparams):
    '''Converts mel spectrogram to waveform using librosa'''
    if hparams.signal_normalization:
        D = _denormalize(mel_spectrogram, hparams)
    else:
        D = mel_spectrogram

    S = _mel_to_linear(_db_to_amp(D + hparams.ref_level_db), hparams)  # Convert back to linear

    if hparams.use_lws:
        processor = _lws_processor(hparams)
        D = processor.run_lws(S.astype(np.float64).T ** hparams.power)
        y = processor.istft(D).astype(np.float32)
        return inv_preemphasis(y, hparams.preemphasis, hparams.preemphasize)
    else:
        return inv_preemphasis(_griffin_lim(S ** hparams.power, hparams), hparams.preemphasis, hparams.preemphasize)

In [ ]:
#signal_normalization == True 경우 
# Whether to normalize mel spectrograms to some predefined range (following below parameters)
D = _denormalize(mel_spectrogram)
#signal_normalization이 아닌경우
D = mel_spectrogram

In [ ]:
S = _mel_to_linear(_db_to_amp(D+hparams.ref_level_db),hparams) ## Convert back to linear


In [ ]:
use_lws=False, #Only used to set as True if using WaveNet, no difference in performance is observed in either cases.

if use_lws:
    processor = 

#### _mel_to_linear

np.linalg.pinv : Calculate the generalized inverse of a matrix using its singular-value decomposition (SVD) and including all large singular values.

In [ ]:
_inv_mel_basis=None
def _mel_to_linear(mel_spectrogram):
	global _inv_mel_basis
	if _inv_mel_basis is None:
		_inv_mel_basis = np.linalg.pinv(_build_mel_basis())
	return np.maximum(1e-10, np.dot(_inv_mel_basis, mel_spectrogram))

In [ ]:
#def _build_mel_basis(hparams):
#    assert hparams.fmax <= hparams.sample_rate // 2
#    return librosa.filters.mel(hparams.sample_rate, hparams.n_fft, n_mels=hparams.num_mels,fmin=hparams.fmin, fmax=hparams.fmax)
#n_fft(2048) : Extra window size is filled with 0 paddings to match this parameter

def _build_mel_basis():
    assert 7600 <= 22050//2
    return librosa.filters.mel(22050,2048,n_mels= 80, fmin=55,fmax=7600)

#### _lws_processor()

In [66]:
def _lws_processor(): #default 코드에는 사용되지 않음
    import lws
    #return lws.lws(hparams.n_fft, get_hop_size(hparams), fftsize=hparams.win_size, mode="speech")
    return lws.lws(2048,get_hop_size(), fftsize=1100, mode="speech")
#win_size : For 22050Hz, 1100 ~= 50 ms (If None, win_size = n_fft) (0.05 * sample_rate)
#lws.lws : Fast spectrogram phase recovery using Local Weighted Sums (LWS)

_lws_processor()
#Only used to set as True if using WaveNet, no difference in performance is observed in either cases.

ValueError: LWS requires that the window shift divides the window length.

In [81]:
def melspectrogram(wav): #(audio.py)
    D = _stft(preemphasis(wav,0.97,True)) #(1025, 802)
    S = _amp_to_db(_linear_to_mel(np.abs(D))) - 20 #(80,802)
melspectrogram(wav)

In [82]:
mel_spectrogram = melspectrogram(wav).astype(np.float32)
mel_spectrogram.shape[1]

AttributeError: 'NoneType' object has no attribute 'astype'